In [2]:
import pathlib
import io
import datetime
import json
import pathlib

import netCDF4
import numpy as np
import pandas as pd
import tqdm

import geojson

In [3]:
data_dir = '~/data/odv'

data_path = pathlib.Path(data_dir).expanduser()

paths = list(
    data_path.glob('*_QC_done_v2.nc')
)

# path = pathlib.Path('~/data/odv/data_from_SDN_2017-11_TS_profiles_non-restricted_med.nc').expanduser()
path = paths[0]
ds  = netCDF4.Dataset(path)


In [9]:
def odvnc2json(path):
    ds = netCDF4.Dataset(path)
    lat = ds.variables['latitude'][:]
    lon = ds.variables['longitude'][:]
    points = np.c_[lon, lat].tolist()
    pts = geojson.MultiPoint(points)
    new_path = path.with_suffix('.json')

    with new_path.open('w') as f:
        geojson.dump(pts, f)
for path in paths:
    odvnc2json(path)


In [37]:
paths[0]

PosixPath('/Users/baart_f/data/odv/data_from_SDN_2015-09_TS_ArcticOcean_QC_done_v2.nc')

In [38]:

def antimeridian_cut(lon):
    """longitudes > 180 -> -360"""
    return np.mod(np.array(lon) + 180, 360) - 180


def odvnc2features(path):
    ds = netCDF4.Dataset(path)
    # slicing in time!
    features = []
    for year in range(2000, 2019):
        t0 = netCDF4.date2num(
            datetime.datetime(year=year, month=1, day=1),
            ds.variables['date_time'].units
        )
        t1 = netCDF4.date2num(
            datetime.datetime(year=year + 1, month=1, day=1),
            ds.variables['date_time'].units
        )

        # ensure that our array is always masked
        date_time = np.ma.masked_array(
            ds.variables['date_time'][:]
        )
        is_in_date = np.logical_and(
            date_time[:] >= t0,
            date_time[:] < t1
        ).data
        if not is_in_date.any():
            # no data, skipping
            continue
        t = np.empty(
            len(date_time[is_in_date]),
            dtype=type(datetime.datetime.now())
        )

        # split nans and notnans makes it much faster
        dtf = np.where(date_time[is_in_date].mask == False)
        dtt = np.where(date_time[is_in_date].mask == True)
        t[dtf] = netCDF4.num2date(
            date_time[is_in_date][dtf],
            ds.variables['date_time'].units
        )
        # do we have any masked values
        if dtt and dtt[0]:
            t[dtt] = netCDF4.num2date(
                date_time[is_in_date][dtt],
                ds.variables['date_time'].units
            )

        # # TODO: slicing through Depth... Hard with this sort of unstructured netcdf.
        # if data['var1'].long_name == "Depth":
        #     depth = None
        # else:
        depth = None

        if 'lat' in ds.variables:
            lat = ds['lat'][is_in_date]
        elif 'latitude' in ds.variables:
            lat = ds['latitude'][is_in_date]
        if 'lon' in ds.variables:
            lon = ds['lon'][is_in_date]
        elif 'longitude' in ds.variables:
            lon = ds['longitude'][is_in_date]

        # find LOCAL_CDI_ID
        for name,  var in ds.variables.items():
            if var.long_name == 'LOCAL_CDI_ID':
                break
        else:
            raise ValueError('LOCAL_CDI_ID not found')
        cdi_id = netCDF4.chartostring(var[is_in_date])

        coordinates = np.c_[
            antimeridian_cut(lon),
            lat
        ].tolist()


        for i, (coordinate, cdi_id_i) in enumerate(zip(coordinates, cdi_id)):
            geometry = geojson.Point(coordinate)
            feature = geojson.Feature(
                id=i,
                geometry=geometry,
                properties={
                    "cdi_id": cdi_id_i,
                    "year": year,
                    "dataset": path.name
                }
            )
            features.append(feature)

    collection = geojson.FeatureCollection(features=features)
    new_path = path.with_name(path.stem).with_suffix('.json')
    with new_path.open('w') as f:
        geojson.dump(collection, f)

In [39]:
for path in paths:
    odvnc2features(path)
    

/Users/baart_f/.virtualenvs/main/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [1]:
json_paths = list(
    data_path.glob('*_QC_done_v2.json')
)
json_paths
def path2region(path):
    import re
    pattern = re.compile('TS_(?P<region>\w+)_QC')
    match = pattern.search(str(json_path))
    region = match['region'].lower()
    return region
    
for i, json_path in enumerate(json_paths):
    region = path2region(json_path)
    !echo mapbox upload siggyf.sdc-ts-{region} {json_path} 
for i, json_path in enumerate(json_paths):
    region = path2region(json_path)
    !echo ~/src/tippecanoe/tippecanoe --layer {region} --name {region} -o {region}.mbtiles -Z 0 -z 8 {json_path} 
    !echo mb-util --image_format=pbf {region}.mbtiles {region}
for i, json_path in enumerate(json_paths):
    region = path2region(json_path)
    mbtiles_path = '{}.mbtiles'.format(region)
    !echo mapbox upload siggyf.sdc-ts-{region}-mb {mbtiles_path}     

NameError: name 'data_path' is not defined

In [31]:
cdi_id_set = {
    feature.properties['cdi_id']
    for feature 
    in collection.features
}


In [ ]:
if 'lat' in ds.variables:
    station_lat = ds['lat'][:]
elif 'latitude' in ds.variables:
    station_lat = ds['latitude'][:]
if 'lon' in ds.variables:
    station_lon = ds['lon'][:]
elif 'longitude' in ds.variables:
    station_lon = ds['longitude'][:]

cdi_ids = netCDF4.chartostring(ds.variables['metavar4'][:])
jsons = []
for i, cdi_id in enumerate(tqdm.tqdm(list(cdi_id_set))):
    
    cdi_id = str(cdi_id)


    # get the first
    idx = cdi_ids == cdi_id
    

    var_names = [
        name
        for name, var
        in ds.variables.items()
        if name.startswith('var') and not '_' in name
    ]

    # add the variables to the list
    variables = {}
    for var_name in var_names:
        var = ds.variables[var_name]
        variables[var.long_name] = np.squeeze(var[idx]).ravel()

    # get metadata
    date_nums = ds.variables['date_time'][idx] 
    
    date_units = ds.variables['date_time'].units
    date = netCDF4.num2date(date_nums.ravel().max(), date_units)
    
    date_nums_expanded = np.zeros_like(var[idx].filled()) + np.atleast_2d(date_nums).T
    dates = netCDF4.num2date(
        date_nums_expanded,
        date_units
    )
    
    variables['Date'] = [date_i.isoformat() for date_i in list(dates.ravel())]

        
    df = pd.DataFrame(data=variables, index=np.arange(np.squeeze(var[idx]).ravel().shape[0]))
    # get rid of missing data
    df = df.dropna(how='all')

    records = json.loads(df.to_json(orient='records'))


    response = {
        "data": records,
        "meta": {
            "date": date.isoformat(),
            "cdi_id": cdi_id
        }
    }
    jsons.append(response)
    if i > 10:
        break


  0%|          | 6/71810 [00:00<1:35:22, 12.55it/s]

In [10]:
heatmap_template = {
    "id": "heatmap",
    "type": "heatmap",
    "source": "sdc-med-profiles",
    "source-layer": "sdc-med-profiles",
    "layout": {},
    "paint": {
        "heatmap-opacity": 1,
        "heatmap-color": [
            "interpolate",
            ["linear"],
            ["heatmap-density"],
            0,
            "rgba(0, 0, 255, 0)",
            0.3,
            "hsla(180, 100%, 50%, 0.49)",
            1,
            "hsl(185, 100%, 100%)"
        ],
        "heatmap-radius": [
            "interpolate",
            ["linear"],
            ["zoom"],
            4,
            1,
            22,
            15
        ]
    }
}
circles_template = {
    "id": "circles",
    "type": "circle",
    "source": "sdc-med-profiles",
    "source-layer": "sdc-med-profiles",
    "layout": {},
    "paint": {
        "circle-color": "hsla(185, 100%, 100%, 0.2)",
        "circle-radius": [
            "interpolate",
            ["linear"],
            ["zoom"],
            6,
            1.5,
            22,
            10
        ]
    }
}
point_template = {
    "id": "point-layer",
    "type": "circle",
    "source": "point-layer",
    "layout": {},
    "paint": {
        "circle-color": "hsla(180, 100%, 80%, 0.49)"
    }
}
text_template = {
    "id": "point-layer-text",
    "type": "symbol",
    "source": "point-layer",
    "layout": {
        "text-field": ["get", "cdi_id"],
        "text-padding": [
            "interpolate",
            ["linear"],
            ["zoom"],
            0,
            2,
            22,
            2
        ]
    },
    "paint": {
        "text-color": "hsl(0, 0%, 100%)",
        "text-opacity": 1
    }
}



In [13]:
import copy


layers = [point_template, text_template]
sources = {}

for json_path in json_paths:
    heatmap = copy.deepcopy(heatmap_template)
    circles = copy.deepcopy(circles_template)
    region = path2region(json_path)
    source = "sdc-ts-{}-mb".format(region)
    
    heatmap['id'] = "heatmap-{}".format(region)
    heatmap['source'] = source
    heatmap['source-layer'] = region
    layers.append(heatmap)

    circles['id'] = "circles-{}".format(region)
    circles['source'] = source
    circles['source-layer'] = region
    layers.append(circles)
        
    sources[source] = {
        "url": "mapbox://siggyf.{}".format(source),
        "type": "vector"
    }

In [14]:
with open('ts-layers.json', 'w') as f:
    json.dump(layers, f, indent=2)
with open('ts-sources.json', 'w') as  f:
    json.dump(sources, f, indent=2)